In [1]:
import os
os.chdir("../../")


In [ ]:
import pandas as pd
import numpy as np
import whisper
import warnings
warnings.filterwarnings("ignore")
from jiwer import wer
import pandas as pd
from datasets import Dataset, Audio
from sklearn.model_selection import train_test_split
import torch
import numpy as np
import librosa
from transformers import AutoProcessor
from torch.utils.data import DataLoader
from datasets import Dataset as HFDataset
import re
import time

### Fonction 

In [3]:
from IPython.display import HTML



def Ecoute_audio(file_path):
  return HTML(f"""
<audio controls>
  <source src="{file_path}" type="audio/mpeg">
  Votre navigateur ne supporte pas la balise audio.
</audio>
""") 

In [4]:
from vosk import Model, KaldiRecognizer
import wave
import json
import whisper

def transcribe(file_path, model, model_name="tiny"):
    
    if model=="whisper":
        
        model = whisper.load_model(model_name)

        result = model.transcribe(file_path)

        result = result["text"]
        return result
    
    elif model=="vosk":
        wf = wave.open(file_path, 'rb')
        model = Model(lang="en-us")
        rec = KaldiRecognizer(model, wf.getframerate())
        rec.SetWords(True)

        full_text = ""

        while True:
            data = wf.readframes(4000)
            if len(data) == 0:
                break
            if rec.AcceptWaveform(data):
                result = json.loads(rec.Result())
                
        full_text = json.loads(rec.Result()).get("text")
        if full_text=="":
            full_text = result.get("text")

        return full_text
    
    elif model=="w2v2":
        df1= df.select([row])
        dataset = Wav2Vec2Dataset(df1, processor, AUDIO_DIR)

        input_values = processor(
            dataset[0]["input_values"],
            sampling_rate=16000,  
            return_tensors="pt",
            padding="longest"
        ).input_values


        logits = model_w2v2(input_values).logits


        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)
        
        return transcription[0]

In [6]:
def evaluate_transcribe(df, model, model_name = "tiny", iter = -1):

    iter= len(df) if iter == -1 else iter
    
    list_score = []
    list_duration = []        
        
        
    if model=="whisper":
        
        
        print(f"Model: {model}/{model_name}")  
        
        model = whisper.load_model(model_name)
        
        
        for i in range(iter):
            
            if i % 3000 == 3000:
                print(f"Processing index: {i}")
                
                
                
            start = time.time()
            
            result = model.transcribe(f'{AUDIO_DIR}/{df["audio"].iloc[i]}')

            end = time.time()

            duration = end - start
            
            
            
            reference = df["transcription"].iloc[i].lower() 
            reference = re.sub(r'[^\w\s]', '', reference)  
            

            hypothesis = result["text"].lower() 
            hypothesis = re.sub(r'[^\w\s]', '', hypothesis)  

            score = wer(reference, hypothesis)

            list_score.append(score)
            list_duration.append(duration)
            
        moyenne_duration = sum(list_duration) / len(list_duration)
        moyenne_score = sum(list_score) / len(list_score)
        print(f"WER moyen: {moyenne_score:.2%}")
        print(f"Durée moyenne de transcription : {moyenne_duration:.2f} secondes")
        
        
    elif model=="vosk":
           
        print(f"Model: {model}")
        model = Model(lang="en-us")

        for i in range(iter):
            
            if i % 3000 == 3000:
                print(f"Processing index: {i}")
                
                
            wf = wave.open(f'{AUDIO_DIR}/{df["audio"].iloc[i]}', 'rb')

                
            rec = KaldiRecognizer(model, wf.getframerate())
            rec.SetWords(True)

            full_text = ""

            start = time.time()

            while True:
                data = wf.readframes(4000)
                if len(data) == 0:
                    break
                if rec.AcceptWaveform(data):
                    result = json.loads(rec.Result())
                    
            
            end = time.time()

            duration = end - start
            
            full_text = json.loads(rec.Result()).get("text")
            if full_text=="":
                full_text = result.get("text")
                
            full_text = re.sub(r'[^\w\s]', '', full_text)  
            hypothesis = full_text.lower()  

            reference = df["transcription"].iloc[i].lower()  
            reference = re.sub(r'[^\w\s]', '', reference)  
            
            score = wer(reference, hypothesis)

            list_score.append(score)
            list_duration.append(duration)
                
        moyenne_duration = sum(list_duration) / len(list_duration)
        moyenne = sum(list_score) / len(list_score)
        
    

        print(f"WER moyen: {moyenne:.2%}")
        print(f"Durée moyenne de transcription : {moyenne_duration:.2f} secondes")
        
        
        
    if model=="w2v2":
            
        
        print(f"Model: {model}")  
        
        df = Dataset.from_pandas(df)
        df = df.cast_column("audio", Audio(decode=False))
        
        for i in range(iter):
            
            if i % 3000 == 3000:
                print(f"Processing index: {i}")
                
                
            start = time.time()
            df1= df.select([i])
            dataset = Wav2Vec2Dataset(df1, processor, AUDIO_DIR)

            input_values = processor(
                dataset[0]["input_values"],
                sampling_rate=16000,  
                return_tensors="pt",
                padding="longest"
            ).input_values


            logits = model_w2v2(input_values).logits


            predicted_ids = torch.argmax(logits, dim=-1)
            transcription = processor.batch_decode(predicted_ids)

            end = time.time()
            duration = end - start

            reference = df1['transcription'][0]
            reference = re.sub(r'[^\w\s]', '', reference).lower()
            

            hypothesis = transcription[0].lower()
            

            score = wer(reference, hypothesis)

            list_score.append(score)
            list_duration.append(duration) 

            
        moyenne_duration = sum(list_duration) / len(list_duration)
        moyenne_score = sum(list_score) / len(list_score)
        print(f"WER moyen: {moyenne_score:.2%}")
        print(f"Durée moyenne de transcription : {moyenne_duration:.2f} secondes")
        
    
    

# Pré Traitement

In [7]:
OUTPUT_CSV = "data/En/processed/Common Voice.csv" 
AUDIO_DIR = "clips_en.wav"

In [8]:
metadata = pd.read_csv(OUTPUT_CSV)

metadata = metadata.dropna(subset=['wav_path', 'sentence'])

metadata = metadata.rename(columns={"wav_path": "audio"})
metadata = metadata.rename(columns={"sentence": "transcription"})

metadata=metadata[["audio","transcription"]]

metadata.head(5)

,audio,transcription
0,common_voice_en_40865211.wav,"With this transition to the big time, the band..."
1,common_voice_en_40865212.wav,Local brothels recruited extra staff to cope w...
2,common_voice_en_40865213.wav,"With Fox on lead vocals, the threesome did two..."
3,common_voice_en_40865214.wav,Miramax requested cuts be made and Christopher...
4,common_voice_en_40865215.wav,The Key allows customers to buy Plusbus for th...


## W2V2

In [9]:
df = Dataset.from_pandas(metadata)
df = df.cast_column("audio", Audio(decode=False))
df[0]

{'audio': {'bytes': None, 'path': 'common_voice_en_40865211.wav'},
 'transcription': 'With this transition to the big time, the band shortened their name to Stabilo.'}

In [10]:
from transformers import AutoProcessor
import os

processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")

In [11]:
import torch
from torch.utils.data import Dataset
import librosa

class Wav2Vec2Dataset(Dataset):
    def __init__(self, hf_dataset, processor, audio_dir):
        self.dataset = hf_dataset
        self.processor = processor
        self.audio_dir = audio_dir

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        example = self.dataset[idx]
        audio_path = os.path.join(self.audio_dir, example["audio"]["path"])
        speech_array, _ = librosa.load(audio_path, sr=16000)

        input_values = self.processor(speech_array, sampling_rate=16000).input_values[0]
        labels = self.processor.tokenizer(example["transcription"]).input_ids

        return {
            "input_values": torch.tensor(input_values),
            "labels": torch.tensor(labels)
        }

In [12]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader
from datasets import Dataset

train_dataset = Wav2Vec2Dataset(df, processor, AUDIO_DIR)


processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model_w2v2 = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
    

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Test

In [13]:
row = 0
AUDIO_PATH = f'{AUDIO_DIR}/{metadata["audio"].iloc[row]}'
file_path = f'../../{AUDIO_DIR}/{metadata["audio"].iloc[row]}'

    
Ecoute_audio(file_path)

In [14]:
reference = metadata["transcription"].iloc[row]
print(reference)

With this transition to the big time, the band shortened their name to Stabilo.


In [15]:
result_whisper = transcribe(file_path=AUDIO_PATH, model="whisper")
print(result_whisper)

 With this transition to the big time, the band shorten their name to stay below.


In [16]:
result_vosk = transcribe(file_path=AUDIO_PATH, model="vosk")
print(result_vosk)

with this transition to the big time the ban shorten their name to stay below


In [17]:
result_w2v2 = transcribe(file_path=AUDIO_PATH, model="w2v2")
print(result_w2v2)

WITH THIS TRANSITION TO THE BIG TIME DID BAN SHORTEN THEIR NAME TO STAY BELOW


# Métrique

In [18]:
reference_test = "Evaluons sa capacité à transcrire."
hypothesis_test = "Evaluon sa capaciter à transcrir."

score = wer(reference_test, hypothesis_test)
print(f"WER: {score:.2%}")

WER: 60.00%


In [19]:
reference = re.sub(r'[^\w\s]', '', reference).lower() # supprime tout sauf lettres/chiffres/espaces + conversion en minuscules

list_result = [result_whisper, result_vosk, result_w2v2]
list_model = ["Whisper", "Vosk", "Wav2Vec2"]

for i in range(len(list_result)):
    
    hypothesis = re.sub(r'[^\w\s]', '', list_result[i]).lower()  
    score = wer(reference, hypothesis)

    print(f"{list_model[i]}/ WER: {score:.2%}")



Whisper/ WER: 21.43%
Vosk/ WER: 28.57%
Wav2Vec2/ WER: 35.71%


# Evaluation

## CommonVoice

In [20]:
OUTPUT_CSV = "data/En/processed/Common Voice.csv" 
AUDIO_DIR = "clips_en.wav"

In [21]:
metadata = pd.read_csv(OUTPUT_CSV)

metadata = metadata.dropna(subset=['wav_path', 'sentence'])

metadata = metadata.rename(columns={"wav_path": "audio"})
metadata = metadata.rename(columns={"sentence": "transcription"})

metadata=metadata[["audio","transcription"]]

metadata.head(5)

,audio,transcription
0,common_voice_en_40865211.wav,"With this transition to the big time, the band..."
1,common_voice_en_40865212.wav,Local brothels recruited extra staff to cope w...
2,common_voice_en_40865213.wav,"With Fox on lead vocals, the threesome did two..."
3,common_voice_en_40865214.wav,Miramax requested cuts be made and Christopher...
4,common_voice_en_40865215.wav,The Key allows customers to buy Plusbus for th...


In [22]:
df = Dataset.from_pandas(metadata)
df = df.cast_column("audio", Audio(decode=False))
df[0]

{'audio': {'bytes': None, 'path': 'common_voice_en_40865211.wav'},
 'transcription': 'With this transition to the big time, the band shortened their name to Stabilo.'}

In [23]:
evaluate_transcribe(df=metadata ,model="whisper", iter=2)

Model: whisper/tiny
WER moyen: 10.71%
Durée moyenne de transcription : 0.60 secondes


In [24]:
evaluate_transcribe(df=metadata ,model="vosk", iter=2)

Model: vosk
WER moyen: 14.29%
Durée moyenne de transcription : 1.17 secondes


In [25]:
evaluate_transcribe(df=metadata ,model="w2v2", iter=2)

Model: w2v2
WER moyen: 17.86%
Durée moyenne de transcription : 0.35 secondes


## LibriSpeech

In [26]:
OUTPUT_CSV = "data/librispeech_dataset_wav.csv" 
AUDIO_DIR = "data"

In [27]:
metadata = pd.read_csv(OUTPUT_CSV)

metadata = metadata.rename(columns={"wav_path": "audio"})

metadata = metadata.dropna(subset=['audio', 'transcription'])

metadata=metadata[["audio","transcription"]]

metadata.head(5)

,audio,transcription
0,LibriSpeech_wav\train-clean-360\100\121669\100...,TOM THE PIPER'S SON
1,LibriSpeech_wav\train-clean-360\100\121669\100...,THE PIG WAS EAT AND TOM WAS BEAT AND TOM RAN C...
2,LibriSpeech_wav\train-clean-360\100\121669\100...,HE NEVER DID ANY WORK EXCEPT TO PLAY THE PIPES...
3,LibriSpeech_wav\train-clean-360\100\121669\100...,BUT HE WAS SO SLY AND CAUTIOUS THAT NO ONE HAD...
4,LibriSpeech_wav\train-clean-360\100\121669\100...,AND THEY LIVED ALL ALONE IN A LITTLE HUT AWAY ...


In [28]:
df = Dataset.from_pandas(metadata)
df = df.cast_column("audio", Audio(decode=False))
df[0]

{'audio': {'bytes': None,
  'path': 'LibriSpeech_wav\\train-clean-360\\100\\121669\\100-121669-0000.wav'},
 'transcription': "TOM THE PIPER'S SON"}

In [29]:
evaluate_transcribe(df=metadata ,model="whisper", iter=2)

Model: whisper/tiny
WER moyen: 31.67%
Durée moyenne de transcription : 0.53 secondes


In [30]:
evaluate_transcribe(df=metadata ,model="vosk", iter=2)

Model: vosk
WER moyen: 19.17%
Durée moyenne de transcription : 1.03 secondes


In [31]:
evaluate_transcribe(df=metadata ,model="w2v2", iter=2)

Model: w2v2
WER moyen: 12.50%
Durée moyenne de transcription : 0.27 secondes


## L2-ARCTIC

In [32]:
OUTPUT_CSV = "data/L2-ARTICLE.csv" 
AUDIO_DIR = "data"

In [33]:
metadata = pd.read_csv(OUTPUT_CSV)

metadata = metadata.dropna(subset=['audio', 'transcription'])
metadata["audio"] = metadata["audio"].str.replace(r"^\.\.\\", "", regex=True)

metadata=metadata[["audio","transcription"]]

metadata.head(5)

,audio,transcription
0,data4\ABA\wav\arctic_a0001.wav,Author of the danger trail Philip Steels etc
1,data4\ABA\wav\arctic_a0002.wav,Not at this particular case Tom apologized Whi...
2,data4\ABA\wav\arctic_a0003.wav,For the twentieth time that evening the two me...
3,data4\ABA\wav\arctic_a0004.wav,Lord but I'm glad to see you again Phil
4,data4\ABA\wav\arctic_a0005.wav,Will we ever forget it


In [34]:
df = Dataset.from_pandas(metadata)
df = df.cast_column("audio", Audio(decode=False))
df[0]

{'audio': {'bytes': None, 'path': 'data4\\ABA\\wav\\arctic_a0001.wav'},
 'transcription': 'Author of the danger trail Philip Steels etc'}

In [35]:
evaluate_transcribe(df=metadata ,model="whisper", iter=2)

Model: whisper/tiny
WER moyen: 50.00%
Durée moyenne de transcription : 0.53 secondes


In [36]:
evaluate_transcribe(df=metadata ,model="vosk", iter=2)

Model: vosk
WER moyen: 43.75%
Durée moyenne de transcription : 1.08 secondes


In [37]:
evaluate_transcribe(df=metadata ,model="w2v2", iter=2)

Model: w2v2
WER moyen: 50.00%
Durée moyenne de transcription : 0.32 secondes


# Conclusion

- Model: W2V2 

WER moyen: 49.61%

Durée moyenne de transcription : 0.54 secondes

- Model: whisper/tiny

WER moyen: 49.04%

Durée moyenne de transcription : 1.32 secondes

- Model: vosk

WER moyen: 50.66%

Durée moyenne de transcription : 1.40 secondes